# Timing

Sometimes, we need to check how long an operation takes. This can be trivially done with the `%%time` cell magic from jupyter notebook

In [ ]:
import numpy as np

In [ ]:
%%time
a = np.random.rand(10000, 1000)
b = np.random.rand(1000, 10000)
c = a@b # a @ b means matrix multiplication

Normally, we are interested in the *wall time*. That is how long the system actually took to run, in the time measured from the clock on your wall. 

# Function fit

A classic activity is, given a set of points (x,y), to find the parameters of a function $y = f(x)$ which describe the points. We can do this easily with Scipy. 

In [ ]:
import numpy as np
import scipy.optimize as spo
import matplotlib.pyplot as plt

First, we will define the function we want to be fitted. In this case, $f(x) = a \exp(b x) + c$.

In [ ]:
def f(x, a, b,c):
    return a*np.exp(b*x) + c

Now, we are going to generate some mock data, so we have something to test with

In [ ]:
a_mock = 300
b_mock = -5
c_mock = 3

x = np.linspace(0, 1, 10)
y = f(x, a_mock, b_mock, c_mock)

In [ ]:
plt.plot(x,y, ".")

Now, we can use the `curve_fit` function from scipy to find the original parameters. This function returns an array with the best fitting parameters, in the same order defined in the function. That is, `opt[0]` = `a`, `opt[1]` = `b`, and `opt[2]` = `c`.

In [ ]:
popt, _ = spo.curve_fit(f, x, y)
a = popt[0]
b = popt[1]
c = popt[2]

And now, lets see the fit

In [ ]:
plt.plot(x,y, ".")

x_fine = np.linspace(0, 1, 100)
plt.plot(x_fine, f(x_fine, a, b, c))

The curve looks a bit too good. Lets make things harder for scipy and add some noise

In [ ]:
a_mock = 300
b_mock = -5
c_mock = 3

x = np.linspace(0, 1, 10)
y = f(x, a_mock, b_mock, c_mock)

In [ ]:
noise = 2*np.random.rand(len(x)) -1 # An array of random numbers between -1 and 1
y += 0.2*noise*y #add +- 20% of the value as noise

plt.plot(x,y, ".")

In [ ]:
popt, _ = spo.curve_fit(f, x, y)
a = popt[0]
b = popt[1]
c = popt[2]

print(f"Found function f(x) = a*exp(b*x) + c with a = {a}, b={b}, c={c}.")


plt.plot(x,y, ".")
x_fine = np.linspace(0, 1, 100)
plt.plot(x_fine, f(x_fine, a, b, c))

As a side note, there are specialized functions to do linear/polynomial fit in Numpy (`polyfit`) and Scipy (`linregress`). If you know your system is one of those two, you probably should use that function instead.

## Task: Use more points in your fitting. How do the fitted parameters approach the real ones?

## Bonus Task: Setting initial values

It is possible that we have a good estimate about the values of the parameters of the funcion. Using this knowledge, the function has better odds of successfully finding values, and it is faster! We can input this guess into the `p0` entry of `curve_fit`. This should be an array, ordered with your initial guess for the value, in the same order as the output `popt`. 

In this task, we revisit the fitting of $f(x)$ with the following considerations:
* $f(\infty) = a \exp(-\infty) + c = a * 0 + c = c$
* $f(0) = a \exp(0) + c = a + c$

Thus, we can use the last point as an estimate for $c$ and the first point $f(0)$  as an estimate for $a + c$. That is $a_0 = c - f(0)$. Estimate $b_0 = -1$ 